In [ ]:
## 每個classroom repo應有dcard-top100.csv語料，若沒有的話可以到這裡另外下載：
## https://github.com/lope-classroom/lab-hw/blob/77f8a80e300297f4be17c708244a17133227c49c/dcard-top100.csv

In [2]:
# 下載ckip
!pip install -U ckip-transformers

# dependencies
import jieba
import jieba.posseg as pseg
import logging
jieba.setLogLevel(logging.INFO)

import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.7 MB/s eta 0:00:00


## 題目1. **比較jieba和CKIP斷詞**
> 本週作業使用同一份dcard-top100語料，分別用jieba及CKIP做斷詞，將斷詞出來的結果做成詞頻表（有word, frequency兩個欄位的dataframe），列印出「前10名」以及「倒數10名」的**二字詞**和頻率。

In [3]:
import pandas as pd
df=pd.read_csv('dcard-top100.csv')
df

,ID,title,content
0,235443715,三更 21歲這年我差點被活活燒死,部分回應在B117 \r\n謝謝各位的留言，我都有看完\r\n好的不好的，我都接受謝謝大家🙇...
1,235442609,超狂學經歷！195公分帥家教徵學生,https://i.imgur.com/REIEzSd.jpg\r\n\r\n身高195公分...
2,235441998,我的模特界時間管理大師前男友,看過這麼多在Dcard、PTT上的感情渣事和創作文\r\n從沒想過如此荒謬像八點檔的事情居然...
3,235441259,豆皮加爆,剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\r\n\r\n沒...
4,235442693,這樣女生該追嗎,已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \r\n如下圖\r\n\r\n\r\nh...
...,...,...,...
95,235444562,主題：💟愛情塔羅💟 他要的感情關係vs 我要的感情關係為何？兩個人的近期發展（曖昧/交往中/...,🤗感謝我們之間的連結帶領你來到這裡！ \r\n\r\n主題：💟愛情塔羅💟 他要的感情關係vs...
96,235441027,#分享 老花古董包 LV 方盒子｜Celine 化妝箱,近期購入的兩款古董包 可能因為是新歡 所以都讓我愛不釋手(๑•̀ •́)و✧\r\n不過正是...
97,235447320,在公車上遇到的超尷尬事件,剛剛在公車上有一個阿伯下車前跟我說：妹妹你很有愛心欸 讓位置給別人 不像有些人都在裝睡\r\...
98,235440749,#大眾占卜 🥺。他現在對我的想法是?他的下一步是什麼?我能為這段關係做什麼樣的努力❣,大家好這裡是金魚🙏\r\n這次的占卜適合斷聯、曖昧、復合的族群\r\n\r\n再上圖之前先置...


In [4]:
df_content=df['content']
df_content

0     部分回應在B117 \r\n謝謝各位的留言，我都有看完\r\n好的不好的，我都接受謝謝大家🙇...
1     https://i.imgur.com/REIEzSd.jpg\r\n\r\n身高195公分...
2     看過這麼多在Dcard、PTT上的感情渣事和創作文\r\n從沒想過如此荒謬像八點檔的事情居然...
3     剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\r\n\r\n沒...
4     已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \r\n如下圖\r\n\r\n\r\nh...
                            ...                        
95    🤗感謝我們之間的連結帶領你來到這裡！ \r\n\r\n主題：💟愛情塔羅💟 他要的感情關係vs...
96    近期購入的兩款古董包 可能因為是新歡 所以都讓我愛不釋手(๑•̀ •́)و✧\r\n不過正是...
97    剛剛在公車上有一個阿伯下車前跟我說：妹妹你很有愛心欸 讓位置給別人 不像有些人都在裝睡\r\...
98    大家好這裡是金魚🙏\r\n這次的占卜適合斷聯、曖昧、復合的族群\r\n\r\n再上圖之前先置...
99    「所以，你要當我女朋友嗎？」我看進她的眼睛，隱約反射出自己的倒影。躺在床上的我們呼吸有些紊亂...
Name: content, Length: 100, dtype: object

In [34]:
#正規表達式
import re

cleaned_sentence=[]

for sentence in df_content:
  matches=re.sub(r'\s+', ' ', re.sub(r'[^\u4e00-\u9fa5]', ' ', sentence).replace('\n', ''))
  cleaned_sentence.append(matches)

show_df = pd.DataFrame(cleaned_sentence)
show_df

,0
0,部分回應在 謝謝各位的留言 我都有看完 好的不好的 我都接受謝謝大家 第三次更新在這邊 這邊...
1,身高 公分的男大生楊承翰在家教社團 文徵學生的文章被網友推爆了 網友們看到他的學經歷及成績...
2,看過這麼多在 上的感情渣事和創作文 從沒想過如此荒謬像八點檔的事情居然會發生在自己的身上 本...
3,剛剛吃小火鍋 跟店員說不要金針菇 怕卡牙縫 於是店員幫我換其他配料 沒想到餐一上桌竟是豆皮加...
4,已經約好見面 到了當天晚上七點半才回 我是被耍了嗎 如下圖
...,...
95,感謝我們之間的連結帶領你來到這裡 主題 愛情塔羅 他要的感情關係 我要的感情關係為何 兩個...
96,近期購入的兩款古董包 可能因為是新歡 所以都讓我愛不釋手 不過正是古董包的緣故 包況總有些不...
97,剛剛在公車上有一個阿伯下車前跟我說 妹妹你很有愛心欸 讓位置給別人 不像有些人都在裝睡 講完...
98,大家好這裡是金魚 這次的占卜適合斷聯 曖昧 復合的族群 再上圖之前先置入我的 大眾占卜集合多...


In [6]:
#用jieba製作出來的「前十名」
from collections import Counter

jieba_list=[]

for sentence in cleaned_sentence:
  sentences=jieba.cut(sentence)
  for w in sentences:
    if len(w)==2:
      jieba_list.append(w)

word_counts=Counter(jieba_list)
jieba_10most_cons=word_counts.most_common(10)

df = pd.DataFrame(jieba_10most_cons,columns=['word','freq'])
df

,word,freq
0,因為,125
1,可以,118
2,真的,118
3,所以,117
4,自己,115
5,我們,106
6,大家,95
7,一個,94
8,時候,75
9,知道,70


In [7]:
#用jieba製作出來的「倒數十名」

df2 = pd.DataFrame.from_dict(word_counts, orient='index', columns=['freq'])
df2 = df2.reset_index().rename(columns={'index': 'word'}) #將原本於index的詞，改為「word」的欄位
df2 = df2.sort_values(['freq'], ascending=True) #由小到大排序
jieba_10last_cons=df2.head(10) #從小到大的排序中，找出前十個
jieba_10last_cons

,word,freq
2321,青春,1
2993,拍起,1
2994,嬌藝,1
2995,實話,1
2997,打爆,1
2998,加深,1
3001,不紅,1
3003,整組,1
3004,妖嬌,1
3005,短頭,1


In [8]:
#CKIP

ws_driver = CkipWordSegmenter(device=0)
pos_driver = CkipPosTagger(device=0)


In [22]:
#檢查是否有欄位有空值；因為 ws_driver（）無法處理空值
nun_deletion=[]
for text in cleaned_sentence:
  if text!=' ':
    nun_deletion.append(text)
    

In [ ]:
#用ckip製作出來的「前十名」

ckip_list=[]

for text in nun_deletion:
  texts = text.strip().splitlines()
  ws_texts = ws_driver(texts)
  pos_texts = pos_driver(ws_texts)

  for sentence, sentence_ws in zip(texts, ws_texts):
    #print(sentence)
    #print('\t'.join(sentence_ws))
    for w in sentence_ws:
      if len(w)==2:
        ckip_list.append(w)

ckip_word_counts=Counter(ckip_list)
jieba_10most_cons=ckip_word_counts.most_common(10)

ckip_df = pd.DataFrame(jieba_10most_cons,columns=['word','freq'])

**👆🏻上方 Tokenization 執行完後的結果段落太長，所以先將執行完後的結果刪除了～👆🏻**

In [31]:
#用ckip製作出來的「前十名」
ckip_df

,word,freq
0,因為,129
1,所以,117
2,可以,115
3,真的,115
4,自己,115
5,我們,110
6,大家,95
7,沒有,95
8,覺得,90
9,時候,73


In [32]:
#用ckip製作出來的「倒數十名」

ckip_df2 = pd.DataFrame.from_dict(ckip_word_counts, orient='index', columns=['freq'])
ckip_df2 = ckip_df2.reset_index().rename(columns={'index': 'word'}) #將原本於index的詞，改為「word」的欄位
ckip_df2 = ckip_df2.sort_values(['freq'], ascending=True) #由小到大排序
ckip_10last_cons=ckip_df2.head(10) #從小到大的排序中，找出前十個
ckip_10last_cons


,word,freq
3843,位子,1
1562,人緣,1
2801,腳尖,1
1564,球經,1
2800,哎唷,1
2799,光芒,1
2798,遠方,1
1561,愛車,1
2797,無意,1
1571,壞壞,1
